In [1]:
    if __name__ == '__main__':
        %reload_ext pidgin

In [2]:
    import ast, typing as t

In [3]:
    class FrontMatter:
        def front_matter(Weave, str: "A string that may have yaml blocks") -> ("A string with the yaml removed", t.Dict):
            if str.startswith('---'):
                if '\n---' in str:
                    data, sep, body = str.lstrip('---').partition('---\n')
                    data = __import__('yaml').safe_load(__import__('io').StringIO(
                        Weave.environment.from_string(data).render({**vars(builtins), **Weave.user_ns})))
                    if isinstance(data, dict): return body, data
            return str, {}

In [4]:
    class Yaml(ast.NodeTransformer):
        def visit_Str(Emojize, node):
            if ':' in node.s:
                if node.s.lstrip().startswith('---') and '\n' in node.s:
                    e = ast.parse("""next(__import__('yaml').safe_load_all(__import__('io').StringIO()))""").body[0].value
                    e.args[0].args[0].args = [node]
                    return e
            return node

In [5]:
    def load_ipython_extension(shell):
        shell.ast_transformers.insert(0, Yaml())

In [6]:
    def unload_ipython_extension(shell):
        for i, object in enumerate(shell.ast_transformers):
            if isinstance(object, Yaml): shell.ast_transformers.pop(i); break

>>> """---
... a: b"""
{'a': 'b'}